In [1]:
!pip install biopython

   ---------------------------------------- 0.0/2.8 MB ? eta -:--:--
   ---------------------------------------- 0.0/2.8 MB ? eta -:--:--
    --------------------------------------- 0.0/2.8 MB 495.5 kB/s eta 0:00:06
   - -------------------------------------- 0.1/2.8 MB 657.6 kB/s eta 0:00:05
   - -------------------------------------- 0.1/2.8 MB 656.4 kB/s eta 0:00:05
   -- ------------------------------------- 0.1/2.8 MB 655.8 kB/s eta 0:00:05
   --- ------------------------------------ 0.2/2.8 MB 942.1 kB/s eta 0:00:03
   ----- ---------------------------------- 0.4/2.8 MB 1.3 MB/s eta 0:00:02
   -------- ------------------------------- 0.6/2.8 MB 1.7 MB/s eta 0:00:02
   ---------- ----------------------------- 0.7/2.8 MB 1.9 MB/s eta 0:00:02
   ------------- -------------------------- 0.9/2.8 MB 2.1 MB/s eta 0:00:01
   --------------- ------------------------ 1.1/2.8 MB 2.3 MB/s eta 0:00:01
   ------------------ --------------------- 1.3/2.8 MB 2.4 MB/s eta 0:00:01
   -------------


[notice] A new release of pip is available: 23.3.1 -> 24.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [21]:
from Bio import SeqIO
fastq_file = "16S_WT_day3_11_SRR2628505_1.fastq"
with open(fastq_file, "r") as handle:
    for i, record in enumerate(SeqIO.parse(handle, "fastq")):
        print(f"ID: {record.id}")
        print(f"Sequence: {record.seq}")
        print(f"Quality: {record.letter_annotations['phred_quality']}")
        if i >= 10:
            break

ID: SRR2628505.45838
Sequence: CCTACGGGGGGCAGCAGTGAGGAATATTGGTCAATGGACGAGAGTCTGAACCAGCCAAGTAGCGTGAAGGATGACTGCCCTATGGGTTGTAAACTTCTTTTATATGGGAATAAAACAGGGTATGCATACCCTCTTGTATGTACCATATGAATAAGGATCGGCTAACTCCGTGCCAGCAGCCGCGGTAATACGGAGGATCCGAGCGTTATCCGGATTTATTGGGTTTAAAGGGAGCGTAGGTGGA
Quality: [37, 37, 37, 37, 37, 38, 38, 38, 38, 38, 38, 38, 38, 38, 38, 39, 39, 39, 39, 39, 39, 39, 39, 39, 39, 39, 39, 39, 39, 39, 39, 39, 39, 39, 39, 39, 39, 39, 39, 38, 38, 38, 38, 38, 39, 39, 39, 39, 39, 39, 39, 39, 39, 39, 39, 38, 39, 39, 39, 39, 39, 39, 39, 38, 38, 38, 38, 38, 39, 39, 39, 39, 39, 39, 39, 39, 39, 39, 39, 39, 39, 39, 39, 39, 39, 39, 39, 38, 38, 38, 39, 39, 39, 39, 39, 39, 39, 39, 39, 39, 39, 39, 39, 39, 39, 39, 39, 39, 39, 38, 38, 38, 38, 38, 38, 38, 38, 38, 38, 38, 38, 38, 38, 38, 38, 38, 38, 38, 38, 38, 38, 38, 38, 38, 38, 38, 38, 38, 38, 38, 38, 38, 38, 38, 38, 37, 37, 37, 37, 37, 37, 37, 37, 37, 37, 37, 37, 37, 37, 37, 37, 37, 37, 37, 37, 37, 37, 37, 37, 37, 37, 37, 37, 37, 37, 37, 37, 37, 37

In [22]:
def extract_kmers(sequence, k):
    kmers = {}
    for i in range(len(sequence) - k + 1):
        kmer = sequence[i:i + k]
        if kmer in kmers:
            kmers[kmer] += 1
        else:
            kmers[kmer] = 1
    return kmers

In [23]:
def parse_dataset_and_get_kmers(filepath, k):
    kmer_frequencies = {}
    with open(filepath, 'r') as file:
        while True:
            header = file.readline().strip()
            if not header:
                break
            sequence = file.readline().strip()
            plus = file.readline().strip()
            quality = file.readline().strip()
            
            kmers = extract_kmers(sequence, k)
            for kmer, count in kmers.items():
                if kmer in kmer_frequencies:
                    kmer_frequencies[kmer] += count
                else:
                    kmer_frequencies[kmer] = count
    return kmer_frequencies

file_path = "16S_WT_day3_11_SRR2628505_1.fastq"
k = 21
kmer_frequencies = parse_dataset_and_get_kmers(file_path, k)

for kmer, frequency in kmer_frequencies.items():
    print(f"{kmer}: {frequency}")

CCTACGGGGGGCAGCAGTGAG: 1841
CTACGGGGGGCAGCAGTGAGG: 1843
TACGGGGGGCAGCAGTGAGGA: 1842
ACGGGGGGCAGCAGTGAGGAA: 1842
CGGGGGGCAGCAGTGAGGAAT: 1843
GGGGGGCAGCAGTGAGGAATA: 1843
GGGGGCAGCAGTGAGGAATAT: 1850
GGGGCAGCAGTGAGGAATATT: 1850
GGGCAGCAGTGAGGAATATTG: 1858
GGCAGCAGTGAGGAATATTGG: 5585
GCAGCAGTGAGGAATATTGGT: 5569
CAGCAGTGAGGAATATTGGTC: 5576
AGCAGTGAGGAATATTGGTCA: 5578
GCAGTGAGGAATATTGGTCAA: 6924
CAGTGAGGAATATTGGTCAAT: 6932
AGTGAGGAATATTGGTCAATG: 6938
GTGAGGAATATTGGTCAATGG: 6944
TGAGGAATATTGGTCAATGGA: 5470
GAGGAATATTGGTCAATGGAC: 5472
AGGAATATTGGTCAATGGACG: 5469
GGAATATTGGTCAATGGACGA: 4398
GAATATTGGTCAATGGACGAG: 4394
AATATTGGTCAATGGACGAGA: 4397
ATATTGGTCAATGGACGAGAG: 4399
TATTGGTCAATGGACGAGAGT: 4382
ATTGGTCAATGGACGAGAGTC: 4383
TTGGTCAATGGACGAGAGTCT: 4387
TGGTCAATGGACGAGAGTCTG: 4388
GGTCAATGGACGAGAGTCTGA: 4389
GTCAATGGACGAGAGTCTGAA: 4389
TCAATGGACGAGAGTCTGAAC: 4387
CAATGGACGAGAGTCTGAACC: 4399
AATGGACGAGAGTCTGAACCA: 4399
ATGGACGAGAGTCTGAACCAG: 4398
TGGACGAGAGTCTGAACCAGC: 4391
GGACGAGAGTCTGAACCAGC